In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import kstest

import warnings
warnings.simplefilter("ignore")

In [ ]:
df = pd.read_excel('resultsА_B.xlsx')
df.head(10)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.version.unique()

In [ ]:
df.version.value_counts()

In [ ]:
plt.figure(figsize=(15, 15))
control_revenue_1 = df['sum_gamerounds'][(df['version'] == 'gate_30') & (df['retention_1'] == 1)]
test_revenue_1 = df['sum_gamerounds'][(df['version'] == 'gate_40') & (df['retention_1'] == 1)]

plt.hist(control_revenue_1, color='r', bins=50, alpha = 0.5)
plt.hist(test_revenue_1, bins=50, alpha = 0.5)

plt.show()

In [ ]:
plt.figure(figsize=(15, 15))
control_revenue_7 = df['sum_gamerounds'][(df['version'] == 'gate_30') & (df['retention_7'] == 1)]
test_revenue_7 = df['sum_gamerounds'][(df['version'] == 'gate_40') & (df['retention_7'] == 1)]

plt.hist(control_revenue_7, color='r', bins=50, alpha = 0.5)
plt.hist(test_revenue_7, bins=50, alpha = 0.5)

plt.show()

In [ ]:
mannwhitneyu(control_revenue_1, test_revenue_1)

In [ ]:
mannwhitneyu(control_revenue_7, test_revenue_7)

In [ ]:
ttest_ind(control_revenue_1, test_revenue_1, equal_var = False)

In [ ]:
ttest_ind(control_revenue_7, test_revenue_7, equal_var = False)

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import *
import matplotlib.pyplot as plt
from tqdm.auto import tqdm #для красивой полоски

plt.style.use('ggplot')

def get_bootstrap(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.mean, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
):
    boot_data = []
    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            len(data_column_1), 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            len(data_column_1), 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1)-statistic(samples_2)) # mean() - применяем статистику
        
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    quants = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
        
    # Визуализация
    _, _, bars = plt.hist(pd_boot_data[0], bins = 50)
    for bar in bars:
        if bar.get_x() <= quants.iloc[0][0] or bar.get_x() >= quants.iloc[1][0]:
            bar.set_facecolor('red')
        else: 
            bar.set_facecolor('grey')
            bar.set_edgecolor('black')
    
    plt.style.use('ggplot')
    plt.vlines(quants,ymin=0,ymax=50,linestyle='--')
    plt.xlabel('boot_data')
    plt.ylabel('frequency')
    plt.title("Histogram of boot_data")
    plt.show()
       
    return {"p_value": p_value}

In [ ]:
get_bootstrap(control_revenue_1, test_revenue_1)

In [ ]:
get_bootstrap(control_revenue_7, test_revenue_7)

Принимая за нулевую гипотезу размещение ворот на 30м уровне, и т-тест, и тест Манна-Уитни, а также метод бутстрепа, дают нам понять, что перемещение ворот на 40й уровень не даёт статистически значимых отличий. Ниже проверим их графической реализацией бутстрепа с kaggle.

In [ ]:
boot_1d = []
boot_7d = []
for i in range(500):
    boot_mean_1 = df.sample(frac=1, replace=True).groupby('version')['retention_1'].mean()
    boot_mean_7 = df.sample(frac=1, replace=True).groupby('version')['retention_7'].mean()
    boot_1d.append(boot_mean_1)
    boot_7d.append(boot_mean_7)
    
boot_1d = pd.DataFrame(boot_1d)
boot_7d = pd.DataFrame(boot_7d)

fig, (ax1,ax2) = plt.subplots(1, 2, sharey=True, figsize=(13,5))

boot_1d.plot.kde(ax=ax1)
ax1.set_xlabel("retantion rate",size=12)
ax1.set_ylabel("number of sample",size=12)
ax1.set_title("1 day retention rate distribution", fontweight="bold",size=14)

boot_7d.plot.kde(ax=ax2)
ax2.set_xlabel("retantion rate",size=12)
ax2.set_title("7 days retention rate distribution", fontweight="bold",size=14)
plt.show()

Распределение retention за один день и за семь дней выше для версии, когда ворота на 30м уровне, а значит тестовую гипотезу стоит отвергнуть.